# IMPORTS
 Import the required libraries for the harmonization

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from snowflake.snowpark.functions import col, count, sum as sum_, countDistinct, collect_set, array_agg, array_distinct, array_size, parse_json, flatten, lit, coalesce, when


# DATA HARMONIZATION
- Joining different tables to create the Analytical Dataset (ADS) for analysis

## 1. DIM_STUDIO
- Table DIM_STUDIO contains the metadata of all the available studios such as location, owner, group, etc
- Filters Applied 
    - STATUS = "Active"
    - ISDOMESTIC = TRUE
    - REGIONNAME = 'MI-01' (Michigan), 'NY-01' (Long Island)
- No aggregation

In [ ]:
studio_dim_df = session.table("ORANGE_ZONE.STUDIO.DIM_STUDIO")

In [ ]:
print("There are a total of",studio_dim_df.count(),"studios in the DIM_STUDIO table")

In [ ]:
studio_dim_df_filtered = studio_dim_df.filter(col("STATUS") == "Active").filter(col("ISDOMESTIC") == 1).filter(col("REGIONNAME").isin(["MI-01","NY-01"]))

In [ ]:
print("After applying the necessary filters, there are about",studio_dim_df_filtered.count(),"studios that fall under",studio_dim_df_filtered.select("F_CODE").distinct().count(),"studio groups")

#### Checking if each F_CODE(Studio Group) belong to one Region

In [ ]:
studio_dim_df_filtered.group_by("F_CODE").agg(countDistinct(col("REGIONNAME")))

## 2. Create the base data - JOINS table
 - JOINS table contains details about the new joinees at STUDIO_ID x CLIENT_ID x PERSON_ID x JOIN_DATE level. This is considered as the base data and other tables are joined with this table to bring additional attributes
 - Filters applied - IS_STUDIO_NEW_JOIN = TRUE
 - Aggregation applied - Rolled up the data to Studio_ID x Date level

In [ ]:
joins_df = session.table("ORANGE_ZONE.MEMBER.JOINS")

In [ ]:
joins_df

In [ ]:
joins_df.count()

In [ ]:
studio_dim_df_filtered.select("SOURCE_MBOSTUDIOID").distinct()

## 3. Merge DIM_STUDIO with JOINS data
 This merge is required to get rid of unnecessary studio_IDs for the analysis

In [ ]:
joins_df_filtered = studio_dim_df_filtered.join(joins_df, studio_dim_df_filtered["SOURCE_MBOSTUDIOID"] == joins_df["MBO_STUDIO_ID"], how = "inner")

In [ ]:
joins_df_filtered.count()

In [ ]:
joins_df_filtered.select("MBO_STUDIO_ID").distinct().count()

In [ ]:
joins_agg_data = joins_df_filtered.filter(joins_df_filtered["IS_STUDIO_NEW_JOIN"] == True).group_by(["MBO_STUDIO_ID","F_CODE","REGIONNAME","JOIN_DATE"]).agg(count(col("MBO_STUDIO_ID")).alias("NO_OF_NEW_JOINEES"))

In [ ]:
joins_agg_data

In [ ]:
joins_agg_data.select("MBO_STUDIO_ID").distinct().count()

In [ ]:
joins_agg_data.count()

## 4. DIM_DATE
- DIM_DATE table contains date and date related features such as start of the month, end of the month, etc
- Filter applied - YEAR >=2022 and YEAR<=2025
- Aggregation applied - 

In [ ]:
date_df = session.table("ORANGE_ZONE.REPORTING.DIM_DATE")

In [ ]:
date_df.count()

In [ ]:
date_df

In [ ]:
merged_data = joins_agg_data.join(date_df.filter(date_df["YEAR"]>=2022).filter(date_df["YEAR"]<=2025), joins_agg_data["JOIN_DATE"] == date_df["DATE"] ,how="right")

In [ ]:
merged_data

In [ ]:
merged_data.count()

## LMD_DAILYSNAPSHOTACTIVITY_LEADS
- Table LMD_DAILYSNAPSHOTACTIVITY_LEADS contains details about number of leads generated through different campaigns at STUDIO_ID x CAMPAIGN_ID x DATE level
- Filters applied
- Aggregation applied

In [ ]:
leads_df = session.table("ORANGE_ZONE.LEAD_DL.LMD_DAILYSNAPSHOTACTIVITY_LEADS")

In [ ]:
leads_df.count()

In [ ]:
leads_df.filter(col("CAMPAIGN_ID").is_null()).count() / leads_df.count()

In [ ]:
leads_df

In [ ]:
#leads_filtered = leads_df.join(date_df.filter(date_df["YEAR"]>=2022).filter(date_df["YEAR"]<=2025), ["DATE"] ,how="right")

In [ ]:
#leads_filtered.count()

In [ ]:
#leads_filtered.filter(col("CAMPAIGN_ID").is_null()).count()

In [ ]:
#leads_filtered.filter(col("CAMPAIGN_TYPE").is_null()).count() / leads_filtered.count()

In [ ]:
#leads_filtered.filter(col("CAMPAIGN_ID").is_null()).agg(sum_(col("LEADS")))

## LMD_DAILYSNAPSHOTACTIVITY_ADS
- Table LMD_DAILYSNAPSHOTACTIVITY_ADS contains details about campaign metrics such as spend, reach, impressions,etc for different campaigns at AD_STUDIOID x CAMPAIGN_ID x DATE level
- Filters applied
- Aggregation applied

In [ ]:
ads_df = session.table("ORANGE_ZONE.LEAD_DL.LMD_DAILYSNAPSHOTACTIVITY_ADS")

In [ ]:
#ads_filtered = ads_df.join(date_df.filter(date_df["YEAR"]>=2022).filter(date_df["YEAR"]<=2025), ["DATE"] ,how="right")

In [ ]:
#ads_filtered.count()

In [ ]:
#ads_filtered


In [ ]:
#ads_filtered.group_by(["AD_STUDIOID"]).agg(sum_(col("SPEND")))

In [ ]:
#ads_filtered.filter(col("CAMPAIGN_ID").is_null()).count()

In [ ]:
#ads_filtered.filter(col("AD_STUDIOID").is_null()).count() / ads_filtered.count()

In [ ]:
ads_df.count()

In [ ]:
#ads_df.join(leads_df, (ads_df["AD_STUDIOID"] == leads_df["SOURCE_STUDIOID"]) & (ads_df["DATE"] == leads_df["DATE"]) & (ads_df["CAMPAIGN_ID"] == leads_df["CAMPAIGN_ID"]) ,how = "right").agg(sum_(col("SPEND")))

## Join LEADS and ADS data

In [ ]:
leads_df.count()

In [ ]:
ads_df = ads_df.with_column_renamed("AD_STUDIOID","SOURCE_STUDIOID")

In [ ]:
leads_ads_df = leads_df.join(ads_df, on = ["SOURCE_STUDIOID","CAMPAIGN_ID","DATE"], how = "left")
#leads_ads_df = ads_df.join(leads_df, (ads_df["AD_STUDIOID"] == leads_df["SOURCE_STUDIOID"]) & (ads_df["DATE"] == leads_df["DATE"]) & (ads_df["CAMPAIGN_ID"] == leads_df["CAMPAIGN_ID"]) ,how = "right")

In [ ]:
leads_ads_df.count()

In [ ]:
leads_ads_df.agg(sum_(col("SPEND")))

In [ ]:
#ads_filtered.group_by(["AD_STUDIOID"]).agg(sum_(col("SPEND")))

In [ ]:
leads_ads_df

In [ ]:
# Filling missing value with "Unknown"
leads_ads_df = leads_ads_df.with_column(
    "CAMPAIGN_TYPE",
    coalesce(col("CAMPAIGN_TYPE"), lit("Unknown"))
)



In [ ]:
agg_leads_ads_df = leads_ads_df.group_by(["MBOID","DATE"]).agg(sum_(col("LEADS")).alias("NO_OF_LEADS"), 
                                                           sum_(when(col("CAMPAIGN_TYPE") == "Local", col("LEADS")).otherwise(0)).alias("LOCAL_CAMPAIGN_LEADS"),
                                                           sum_(when(col("CAMPAIGN_TYPE") == "Regional", col("LEADS")).otherwise(0)).alias("REGIONAL_CAMPAIGN_LEADS"),
                                                           sum_(when(col("CAMPAIGN_TYPE") == "National", col("LEADS")).otherwise(0)).alias("NATIONAL_CAMPAIGN_LEADS"),
                                                           sum_(when(col("CAMPAIGN_TYPE") == "Non_Web", col("LEADS")).otherwise(0)).alias("NON_WEB_LEADS"),
                                                           sum_(when(col("CAMPAIGN_TYPE") == "Other", col("LEADS")).otherwise(0)).alias("OTHER_CAMPAIGN_LEADS"),
                                                           sum_(when(col("CAMPAIGN_TYPE") == "Unknown", col("LEADS")).otherwise(0)).alias("UNKNOWN_CAMPAIGN_LEADS"), 
                                                           sum_(col("INTRO_BOOKINGS")).alias("INTRO_BOOKINGS"), 
                                                           sum_(col("INTROS_TAKEN")).alias("INTROS_TAKEN"), 
                                                           sum_(col("MISSED_GUESTS")).alias("MISSED_GUESTS"), 
                                                           sum_(col("CLOSES")).alias("CLOSES"), 
                                                           countDistinct(col("CAMPAIGN_ID")).alias("NO_OF_ACTIVE_CAMPAIGNS"),
                                                           sum_(col("CLICKS")).alias("NO_OF_CLICKS"),
                                                           sum_(col("IMPRESSIONS")).alias("NO_OF_IMPRESSIONS"),
                                                           sum_(col("SPEND")).alias("TOTAL_SPEND"),
                                                           sum_(col("IN_PLATFORM_VIEWTHRU_CAPTURES")).alias("IN_PLATFORM_VIEWTHRU_CAPTURES"),
                                                           sum_(col("WEBSITE_VIEWTHRU_CAPTURES")).alias("WEBSITE_VIEWTHRU_CAPTURES"),
                                                           sum_(col("IN_PLATFORM_CLICKTHRU_CAPTURES")).alias("IN_PLATFORM_CLICKTHRU_CAPTURES"),
                                                           sum_(col("WEBSITE_CLICKTHRU_CAPTURES")).alias("WEBSITE_CLICKTHRU_CAPTURES"),
                                                           sum_(col("WEBSITE_VIEWTHRU_BOOKINGS")).alias("WEBSITE_VIEWTHRU_BOOKINGS"),
                                                           sum_(col("WEBSITE_CLICKTHRU_BOOKINGS")).alias("WEBSITE_CLICKTHRU_BOOKINGS"),
                                                           sum_(col("VIDEO_VIEWS")).alias("VIDEO_VIEWS"),
                                                           sum_(col("WEBSITE_VISITS")).alias("WEBSITE_VISITS"),    
                                                          )

In [ ]:
agg_leads_ads_df

In [ ]:
leads_ads_df = leads_ads_df.join(date_df.filter(date_df["YEAR"]>=2022).filter(date_df["YEAR"]<=2025), ["DATE"] ,how="left")

In [ ]:
# Filling Unknown for 3% of missing  campaign type
#leads_ads_df = leads_ads_df.na.fill({"CAMPAIGN_TYPE","Unknown"})
#leads_ads_df = leads_ads_df.with_column(
#    "CAMPAIGN_TYPE",
#    coalesce(col("CAMPAIGN_TYPE"), lit("Unknown"))
#)

In [ ]:
#leads_ads_df_with_fcode = leads_ads_df.join(studio_dim_df_filtered.select(["SOURCE_MBOSTUDIOID","F_CODE"]), studio_dim_df_filtered["SOURCE_MBOSTUDIOID"] == leads_ads_df["MBOID"], how = "inner")

In [ ]:
#campaign_data_fcode_level = leads_ads_df_with_fcode.group_by(["F_CODE","START_OF_WEEK","CAMPAIGN_TYPE"]).agg(sum_(col("LEADS")).alias("LEADS")).pivot(
#    pivot_col = "CAMPAIGN_TYPE",
#    #values="LEADS"
#).sum("LEADS")

In [ ]:
#campaign_data_fcode_level


In [ ]:
#org_cols = campaign_data_fcode_level.columns
#new_cols = [col.replace('"', '').replace("'","") for col in org_cols]
#for old_col, new_col in zip(org_cols, new_cols):
#    campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed(old_col, new_col)

In [ ]:
#campaign_data_fcode_level

In [ ]:
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed('LOCAL',"LOCAL_CAMPAIGN_LEADS")
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed("NATIONAL","NATIONAL_CAMPAIGN_LEADS")
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed("NON_WEB","NON_WEB_CAMPAIGN_LEADS")
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed("OTHER","OTHER_CAMPAIGN_LEADS")
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed("REGIONAL","REGIONAL_CAMPAIGN_LEADS")
#campaign_data_fcode_level = campaign_data_fcode_level.with_column_renamed("UNKNOWN","UNKNOWN_CAMPAIGN_LEADS")

In [ ]:
#campaign_data_fcode_level

In [ ]:
merged_data2 = merged_data.join(agg_leads_ads_df, (merged_data["MBO_STUDIO_ID"] == agg_leads_ads_df["MBOID"]) & (merged_data["JOIN_DATE"] == agg_leads_ads_df["DATE"]), how = "left")

In [ ]:
merged_data2

In [ ]:
merged_data2.select("MBO_STUDIO_ID").distinct().count()

In [ ]:
#lead_dim_df = session.table("ORANGE_ZONE.LEAD_DL.LEAD_CAPTURE_DIM")

In [ ]:
#lead_dim_df

In [ ]:
#lead_dim_df.group_by("CAMPAIGN_TYPE").agg(count(col("CAMPAIGN_ID")))

In [ ]:
#ads_df.count()

In [ ]:
#ads_filtered.group_by("AD_STUDIOID").agg(sum_(col("SPEND")))

In [ ]:
#lead_dim_df.count()

In [ ]:
#lead_dim_df.select(["CAMPAIGN_ID","CAMPAIGN_TYPE"]).drop_duplicates().filter(col("CAMPAIGN_ID") == "23856229410160147")#group_by("CAMPAIGN_ID").agg(count(col("CAMPAIGN_ID")))

In [ ]:
#ads_filtered.join(lead_dim_df.select(["CAMPAIGN_ID","CAMPAIGN_TYPE"]).drop_duplicates("CAMPAIGN_ID"),["CAMPAIGN_ID"], how = "left").group_by(["CAMPAIGN_TYPE","AD_STUDIOID"]).agg(sum_(col("SPEND")))

## MBO_REVENUE_BY_LINE_ITEM

In [ ]:
#rev_df = session.table("ORANGE_ZONE.REVENUE.MBO_REVENUE_BY_LINE_ITEM")

In [ ]:
#rev_df

In [ ]:
#rev_df.group_by("MBO_CLIENT_ID").agg(countDistinct(col("PERSON_ID")))

In [ ]:
#joins_df_filteredv2 = joins_df_filtered.filter(col("IS_STUDIO_NEW_JOIN") == True)

In [ ]:
#joins_df_filteredv2.group_by("MEMBERSHIP_TYPE").agg(count(col("MEMBERSHIP_TYPE")).alias("NO_OF_NEW_JOINEES"))

In [ ]:
#joins_df_filteredv2

In [ ]:
#rev_df_filtered = joins_df_filteredv2.join(rev_df,(joins_df_filteredv2["MBO_STUDIO_ID"] == rev_df["MBO_STUDIO_ID"]) & (joins_df_filteredv2["MBO_CLIENT_ID"] == rev_df["MBO_CLIENT_ID"]) & (joins_df_filteredv2["MBO_SALE_ID"] == rev_df["MBO_SALE_ID"]) ,how = "left")

In [ ]:
#rev_df_filtered.count()

In [ ]:
#rev_df_filtered

## Rolling up to Studio group and week level

In [ ]:
final_df = merged_data2.filter(col("F_CODE").is_not_null()).group_by(["F_CODE","REGIONNAME","START_OF_WEEK"]).agg(
    sum_(col("NO_OF_NEW_JOINEES")).alias("NO_OF_NEW_JOINEES"),
    sum_(col("NO_OF_LEADS")).alias("TOTAL_LEADS"),
    sum_(col("LOCAL_CAMPAIGN_LEADS")).alias("LOCAL_CAMPAIGN_LEADS"),
   sum_(col("REGIONAL_CAMPAIGN_LEADS")).alias("REGIONAL_CAMPAIGN_LEADS"),
sum_(col("NATIONAL_CAMPAIGN_LEADS")).alias("NATIONAL_CAMPAIGN_LEADS"),
sum_(col("NON_WEB_LEADS")).alias("NON_WEB_LEADS"),
sum_(col("OTHER_CAMPAIGN_LEADS")).alias("OTHER_CAMPAIGN_LEADS"),
sum_(col("UNKNOWN_CAMPAIGN_LEADS")).alias("UNKNOWN_CAMPAIGN_LEADS"),
    sum_(col("INTRO_BOOKINGS")).alias("INTRO_BOOKINGS"), 
    sum_(col("INTROS_TAKEN")).alias("INTROS_TAKEN"), 
    sum_(col("MISSED_GUESTS")).alias("MISSED_GUESTS"), 
    sum_(col("CLOSES")).alias("CLOSES"),
    sum_(col("TOTAL_SPEND")).alias("TOTAL_SPEND"),
    sum_(col("NO_OF_CLICKS")).alias("NO_OF_CLICKS"),
    sum_(col("NO_OF_IMPRESSIONS")).alias("NO_OF_IMPRESSIONS"),
    sum_(col("NO_OF_ACTIVE_CAMPAIGNS")).alias("NO_OF_ACTIVE_CAMPAIGNS"),
    sum_(col("IN_PLATFORM_VIEWTHRU_CAPTURES")).alias("IN_PLATFORM_VIEWTHRU_CAPTURES"),
   sum_(col("WEBSITE_VIEWTHRU_CAPTURES")).alias("WEBSITE_VIEWTHRU_CAPTURES"),
   sum_(col("IN_PLATFORM_CLICKTHRU_CAPTURES")).alias("IN_PLATFORM_CLICKTHRU_CAPTURES"),
   sum_(col("WEBSITE_CLICKTHRU_CAPTURES")).alias("WEBSITE_CLICKTHRU_CAPTURES"),
   sum_(col("WEBSITE_VIEWTHRU_BOOKINGS")).alias("WEBSITE_VIEWTHRU_BOOKINGS"),
   sum_(col("WEBSITE_CLICKTHRU_BOOKINGS")).alias("WEBSITE_CLICKTHRU_BOOKINGS"),
   sum_(col("VIDEO_VIEWS")).alias("VIDEO_VIEWS"),
   sum_(col("WEBSITE_VISITS")).alias("WEBSITE_VISITS"), 
countDistinct(col("MBO_STUDIO_ID")).alias("NO_OF_ACTIVE_STUDIOS")
).sort(["F_CODE","START_OF_WEEK"])

## Export ADS

In [ ]:
#final_df = final_df.join(date_df.filter(date_df["YEAR"]>=2022).filter(date_df["YEAR"]<=2025).select(["DATE","WEEK_OF_MONTH","MONTH_END","QUARTER_END","YEAR_END"]), final_df["START_OF_WEEK"] == date_df["DATE"] ,how="left")

In [ ]:
final_df

In [ ]:
final_df.count()

In [ ]:
final_df.write.mode("overwrite").save_as_table("MY_TEMP_ADS_TABLE_V3")

In [ ]:
#session.sql("""COPY INTO @MY_CSV_STAGE_TEST/ads_data/
#FROM MY_TEMP_ADS_TABLE""").collect()

## Baseline model

In [ ]:
# import pandas as pd
# from prophet import Prophet
# from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# df = final_df.filter(col("F_CODE") == 227).to_pandas()

In [ ]:
# df

In [ ]:
# df = df.rename(columns={'START_OF_WEEK': 'ds', 'NO_OF_NEW_JOINEES': 'y'})
# df = df.sort_values('ds')

In [ ]:
# df

In [ ]:
# train = df.iloc[:-12]   # use all but last 8 weeks for training
# test = df.iloc[-12:]  

In [ ]:
# model = Prophet()
# model.fit(train)

In [ ]:
# future = model.make_future_dataframe(periods=len(test), freq='W')
# forecast = model.predict(future)

In [ ]:
# forecast["ds"] = forecast["ds"].astype('str')

In [ ]:
# df.dtypes

In [ ]:
# pred = forecast[['ds', 'yhat']].merge(df, on='ds', how='left')
# pred_test = pred.iloc[-len(test):]  # last 12 weeks

In [ ]:
# pred

In [ ]:
# a = pred_test["yhat"]

In [ ]:
# b = test["y"]

In [ ]:
# mape = mean_absolute_percentage_error(a,b) * 100
# print(f"MAPE: {mape:.2f}%")

In [ ]:
# df = session.table("ORANGE_ZONE.MEMBER.JOINS")

In [ ]:
# df.group_by(["MBO_CLIENT_ID"]).agg(countDistinct(col("PERSON_ID")).alias("#UNIQUE_PERSONS"))

In [ ]:
# df.filter(col("MBO_CLIENT_ID") == 100048270)

In [ ]:
# df.group_by(["PERSON_ID"]).agg(countDistinct(col("MBO_CLIENT_ID")).alias("#UNIQUE_MEMBERS"))

In [ ]:
# rev = session.table("ORANGE_ZONE.REVENUE.MBO_REVENUE_BY_LINE_ITEM")

In [ ]:
# rev

In [ ]:
# rev.group_by("MBO_PRODUCT_ID").agg(countDistinct(col("REVENUE_TYPE_V2")))

In [ ]:
# rev.filter(col("MBO_PRODUCT_ID") == 119775).select("REVENUE_TYPE_V2").distinct()